In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog
import os
import matplotlib
matplotlib.use('TkAgg')

# --- Step 1: Choose and Load NPZ File ---
Tk().withdraw()
file_path = filedialog.askopenfilename(filetypes=[("NPZ files", "*.npz")], title="Select .npz file")
if not file_path:
    print("Error! No (or wrong) file selected!")
    exit()

filename = os.path.basename(file_path)
Condition = os.path.splitext(filename)[0]
data = np.load(file_path, allow_pickle=True)

# --- Step 2: Extract Variables from .npz ---
Final_Loc_Blinking_Corrected = data['LocalizationsFinal']
Final_Frame_Blinking_Corrected = data['Frame_Information']
addonarray = data['addonarray']
cut1array = data['cut1array']
cut2array = data['cut2array']
cut3array = data['cut3array']
Came_from_image = data['Came_from_image']

# --- Step 3: Initialize Output Structures ---
max_img_idx = int(np.max(Came_from_image)+1)
LocalizationsFinal = [[] for _ in range(max_img_idx)]
Frame_Information = [[] for _ in range(max_img_idx)]
Cutsstore = []

# --- Step 4: Recombine Localizations ---
for ksu in range(len(cut1array)):
    print(f"Processing set {ksu + 1}/{len(cut1array)}")

    X1 = Final_Loc_Blinking_Corrected[ksu][:, 0]
    X2 = Final_Loc_Blinking_Corrected[ksu][:, 1]
    X3 = Final_Loc_Blinking_Corrected[ksu][:, 2]
    X4 = Final_Frame_Blinking_Corrected[ksu]

    addon = addonarray[ksu]
    cut1 = cut1array[ksu]
    cut2 = cut2array[ksu]
    cut3 = cut3array[ksu]
    if cut3[1] == 0:
        cut3 = [-1000, 1000]

    # Determine indices using boundary conditions
    Cutsstore_np = np.array(Cutsstore)
    if Cutsstore:
        ssde = np.subtract([cut1[0], cut2[0]], Cutsstore_np)
        ssde2 = np.subtract([cut1[1], cut2[1]], Cutsstore_np)

        if not np.any(np.sum(ssde, axis=1) == 0) and not np.any(np.sum(ssde2, axis=1) == 0):
            IND = np.where((X1 >= cut1[0]) & (X1 <= cut1[1]) &
                           (X2 >= cut2[0]) & (X2 <= cut2[1]) &
                           (X3 >= cut3[0]) & (X3 <= cut3[1]))[0]
        elif not np.any(np.sum(ssde, axis=1) == 0):
            IND = np.where((X1 >= cut1[0]) & (X1 < cut1[1]) &
                           (X2 >= cut2[0]) & (X2 < cut2[1]) &
                           (X3 >= cut3[0]) & (X3 <= cut3[1]))[0]
        elif not np.any(np.sum(ssde2, axis=1) == 0):
            IND = np.where((X1 > cut1[0]) & (X1 <= cut1[1]) &
                           (X2 > cut2[0]) & (X2 <= cut2[1]) &
                           (X3 >= cut3[0]) & (X3 <= cut3[1]))[0]
        else:
            IND = np.where((X1 > cut1[0]) & (X1 < cut1[1]) &
                           (X2 > cut2[0]) & (X2 < cut2[1]) &
                           (X3 >= cut3[0]) & (X3 <= cut3[1]))[0]
    else:
        IND = np.where((X1 >= cut1[0]) & (X1 <= cut1[1]) &
                       (X2 >= cut2[0]) & (X2 <= cut2[1]) &
                       (X3 >= cut3[0]) & (X3 <= cut3[1]))[0]

    Cutsstore.append([cut1[1], cut2[1]])
    Cutsstore.append([cut1[0], cut2[0]])

    # Plot (3D)
    # fig = plt.figure(1)
    # ax = fig.add_subplot(projection='3d')
    # ax.scatter(X1[IND], X2[IND], X3[IND], c=X4[IND], s=10, cmap='jet')
    # plt.axis('equal')
    # plt.pause(0.5)
    # plt.clf()

    idx = int(Came_from_image[ksu])
    LocalizationsFinal[idx].extend(np.column_stack((X1[IND], X2[IND], X3[IND])).tolist())
    Frame_Information[idx].extend(X4[IND].tolist())

# --- Step 5: Convert to NumPy arrays & Save ---
Final_Loc_Blinking_Corrected = [np.array(loc) for loc in LocalizationsFinal]
Final_Frame_Blinking_Corrected = [np.array(frames) for frames in Frame_Information]

np.savez(f'Combined_Final_{Condition}.npz',
         Final_Loc_Blinking_Corrected=np.array(Final_Loc_Blinking_Corrected, dtype=object),
         Final_Frame_Blinking_Corrected=np.array(Final_Frame_Blinking_Corrected, dtype=object))

print(f"Saved to Combined_Final_{Condition}.npz")

# --- Step 6: Final 2D Plot ---
plt.figure()
plt.scatter(Final_Loc_Blinking_Corrected[0][:, 0],
            Final_Loc_Blinking_Corrected[0][:, 1],
            c=Final_Frame_Blinking_Corrected[0],
            s=20, cmap='jet')
plt.axis('equal')
plt.colorbar()
plt.title("Recombined Localizations (Image 1)")
plt.show()

# # Create the interactive figure
# fig, ax = plt.subplots()

# # Plot the data
# sc = ax.scatter(Final_Loc_Blinking_Corrected[0][:, 0],
#                 Final_Loc_Blinking_Corrected[0][:, 1],
#                 c=Final_Frame_Blinking_Corrected[0],
#                 s=20, cmap='jet')

# # Format the plot
# ax.set_aspect('equal')
# plt.colorbar(sc, ax=ax)
# plt.title("Recombined Localizations (Image 1)")

# # Show interactive window
# plt.show()

Processing set 1/54
Processing set 2/54
Processing set 3/54
Processing set 4/54
Processing set 5/54
Processing set 6/54
Processing set 7/54
Processing set 8/54
Processing set 9/54
Processing set 10/54
Processing set 11/54
Processing set 12/54
Processing set 13/54
Processing set 14/54
Processing set 15/54
Processing set 16/54
Processing set 17/54
Processing set 18/54
Processing set 19/54
Processing set 20/54
Processing set 21/54
Processing set 22/54
Processing set 23/54
Processing set 24/54
Processing set 25/54
Processing set 26/54
Processing set 27/54
Processing set 28/54
Processing set 29/54
Processing set 30/54
Processing set 31/54
Processing set 32/54
Processing set 33/54
Processing set 34/54
Processing set 35/54
Processing set 36/54
Processing set 37/54
Processing set 38/54
Processing set 39/54
Processing set 40/54
Processing set 41/54
Processing set 42/54
Processing set 43/54
Processing set 44/54
Processing set 45/54
Processing set 46/54
Processing set 47/54
Processing set 48/54
P

In [3]:
len(Final_Loc_Blinking_Corrected[0])

372795